In [1]:
import sys
import os
import itertools

import numpy as np
import matplotlib.pyplot as plt

import scipy.optimize as spopt
import scipy.integrate as spi
import scipy.stats as sps

import tensorflow as tf
import pandas as pd

sys.path.append("../../ndsvae/")
import ndsvae as ndsv

sys.path.append("../")
import util
import dsa

%matplotlib inline

In [2]:
config = "ns_3_mreg_3_msub_0_nf_32"
conn = "linw"
preproc = "dicer"
modelname = "AB"

ds = ndsv.Dataset.from_file(f"../run/hcp/hcp100_{conn}_{preproc}/dataset.npz")
run = util.select_run_fc(f"hcp100_{conn}_{preproc}", modelname, config, [0,1], "hcp")
direc = f"../run/hcp/hcp100_{conn}_{preproc}/model{modelname}/{config}/run{run:02d}"
params = util.load_params(os.path.join(direc, "parameters"), np.r_[:100])

model = util.get_model(modelname, config, ds)
model.load_weights(f"../run/hcp/hcp100_{conn}_{preproc}/model{modelname}/{config}/run{run:02d}/fit/model")

2022-08-29 19:17:51.687826: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-29 19:17:51.714422: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 3399905000 Hz
2022-08-29 19:17:51.716770: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55e42e5f35f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-08-29 19:17:51.716876: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-08-29 19:17:51.717368: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
nsub, nreg, _, nt = ds.y.shape

In [4]:
def deduplicate(xs, threshold):
    xds = []    
    for x in xs:  
        if np.any([(np.linalg.norm(x - xd) < threshold) for xd in xds]):
            continue
        else:
            xds.append(x)            
    return np.array(xds)


def find_fixed_points(model, w, thetareg, thetasub, us=0., n=10, init_range=(-2,2), threshold=1e-4):
    nreg = model.nreg
    
    assert thetareg.shape == (nreg, model.mreg)
    assert thetasub.shape == (model.msub,)
    
    x0 = np.random.uniform(init_range[0], init_range[1], size=(n, nreg * model.ns))
    Ap = model.Ap.numpy()
    bp = model.bp.numpy()
    w = w.astype('float32')
    
    has_network = int(model.source_model.network_input)
    has_shared  = int(model.source_model.shared_input)    
    input_size = (model.ns + model.mreg + model.msub + has_network + has_shared)
    
    xaugf = np.zeros((nreg, input_size))
    xaugf[:, model.ns:model.ns+model.mreg] = thetareg
    xaugf[:, model.ns+model.mreg:model.ns+model.mreg+model.msub] = thetasub
    if has_shared: xaugf[:, -1] = us    
    
    def f(x):
        x = x.reshape((nreg, model.ns))
        y = (np.dot(Ap, x.T) + bp)[0,:]
        u = np.dot(w, y)
        
        xaug = np.copy(xaugf)
        xaug[:, :model.ns] = x
        if has_network:
            xaug[:, -has_network-has_shared] = u
        
        fx = model.source_model.f(xaug).numpy().reshape(nreg * model.ns)
        return fx
   
    @tf.function
    def jacobian(x):
        x = tf.cast(x, 'float32')
        with tf.GradientTape(persistent=True, watch_accessed_variables=False) as g:
            g.watch(x)
            xr = tf.reshape(x, (nreg, model.ns))
            y = tf.tensordot(Ap, xr, axes=[[1], [1]])[0] + bp
            u = tf.linalg.matvec(w, y)            

            if not has_network:
                xaug = tf.concat([xr, xaugf[:, model.ns:]], axis=1)
            else:
                xaug = tf.concat([xr,
                                  xaugf[:, model.ns:model.ns+model.mreg+model.msub],
                                  u[:,None],
                                  xaugf[:, model.ns+model.mreg+model.msub+has_network:],
                                 ], axis=1)
        
            fx = model.source_model.f(xaug)
            fx = tf.reshape(fx, (nreg*model.ns,))
                        
        jac = g.jacobian(fx, x)
        return jac                   
        
    xs = []
    fails = 0
    for i in range(n):
        sol = spopt.root(f, x0[i], jac=jacobian, method='hybr', tol=1e-6)

        if not sol.success:
            fails += 1
        else:
            xs.append(sol.x)
        
    xs = np.array(xs)    
    xs = deduplicate(xs, threshold)
    xs = xs.reshape(-1, nreg, model.ns)
    
    # Evaluate stability
    stability = np.zeros(len(xs), dtype=bool)
    eigvals = np.zeros((len(xs), nreg*model.ns), dtype=complex)
    eigvecs = np.zeros((len(xs), nreg*model.ns, nreg*model.ns), dtype=complex)
    
    for i, x in enumerate(xs):
        jac = jacobian(np.reshape(x, nreg*model.ns)).numpy()
        evals, evecs = np.linalg.eig(jac)
        order = np.argsort(-evals.real)
        eigvals[i] = evals[order]
        eigvecs[i] = evecs[:,order]
        stability[i] = np.all(np.real(eigvals[i]) < 0.)
    
    return xs, stability, eigvals, eigvecs

In [5]:
def is_converging(x, t, nsteps, threshold):
    nsteps = min(len(t), nsteps)
    dist = np.linalg.norm(x[:,:,-nsteps:] - x[:,:,[-1]], axis=1)
    converging = np.zeros(x.shape[0], dtype=bool)
    
    for i in range(x.shape[0]):
        if np.all(dist[i] < threshold):
            converging[i] = True
        else:
            last_above = np.where(dist[i] >= threshold)[0][-1]
            dist_above = dist[i,:last_above+1]
            converging[i] = np.all(dist_above[:-1] > dist_above[1:])
            
    return converging
    
    
def is_periodic(x, t, threshold, min_period=0.):
    n = len(t)
    max_offset = int(len(t) / 4)
    
    diff = np.zeros((max_offset, n-max_offset))
    for i in range(n-max_offset):
        diff[:,i] = np.linalg.norm(x[:,[i]] - x[:,i:i+max_offset], axis=0)
    
    min_ind = np.argwhere(t > t[0] + min_period)[0][0]
    candidate_ind = np.where(np.max(diff[min_ind:,:], axis=1) < threshold)[0][0] + min_ind
    
    if np.max(diff[candidate_ind,:]) < threshold:
        return True, t[candidate_ind] - t[0]
    else:
        return False, np.nan


def find_attractors(model, w, thetareg, thetasub, us=0., n=10, init_range=(-2,2), T1=400, T2=1200):
    
    nreg = model.nreg    
    assert thetareg.shape == (nreg, model.mreg)
    assert thetasub.shape == (model.msub,)
    
    x0 = np.random.uniform(init_range[0], init_range[1], size=(n, nreg * model.ns))
    Ap = model.Ap.numpy()
    bp = model.bp.numpy()
    w = w.astype('float32')
    
    has_network = int(model.source_model.network_input)
    has_shared  = int(model.source_model.shared_input)    
    input_size = (model.ns + model.mreg + model.msub + has_network + has_shared)
    
    xaugf = np.zeros((nreg, input_size))
    xaugf[:, model.ns:model.ns+model.mreg] = thetareg
    xaugf[:, model.ns+model.mreg:model.ns+model.mreg+model.msub] = thetasub
    if has_shared: xaugf[:, -1] = us    
    
    def f(t, x):
        x = x.reshape((nreg, model.ns))
        y = (np.dot(Ap, x.T) + bp)[0,:]
        u = np.dot(w, y)
        
        xaug = np.copy(xaugf)
        xaug[:, :model.ns] = x
        if has_network:
            xaug[:, -has_network-has_shared] = u
        
        fx = model.source_model.f(xaug).numpy().reshape(nreg * model.ns)
        return fx
    
    # Simulate all
    xs = []
    ts = []
    
    converging = np.zeros(n, dtype=bool)    
    periodic = np.zeros(n, dtype=bool)
    period = np.full(n, np.nan)    
    
    for i in range(n):
        sol = spi.solve_ivp(f, (0, T1), x0[i], method='RK45', max_step=1)
        if not sol.success:
            raise ValueError(message)
    
        x = np.reshape(sol.y, (nreg, model.ns, -1))
        t = sol.t
    
        converging[i] = is_converging(np.reshape(x, (1, nreg*model.ns, len(t))), t, int(0.1*len(t)), 1e-3)[0]
        
        if not converging[i]:
            sol2 = spi.solve_ivp(f, (0, T2), x0[i], method='RK45', max_step=0.5)            
            if not sol2.success:
                raise ValueError(message)
    
            x = np.reshape(sol2.y, (nreg, model.ns, -1))
            t = sol2.t
            
            # Recheck convergence using this detailed simulation            
            converging[i] = is_converging(np.reshape(x, (1, nreg*model.ns, len(t))), t, int(0.1*len(t)), 1e-3)[0]
            mask = t > T1
            if not converging[i]:
                periodic[i], period[i] = is_periodic(sol2.y[:,mask], sol2.t[mask], 0.1, min_period=2.)
                
        ts.append(t)
        xs.append(x)
    
    return (ts, xs, converging, periodic, period)

In [4]:
nsubs = 100
nrand = 1

rows_fp = []
rows_at = []
fixed_points = []
eigenvalues = []
thetareg = np.zeros((nsubs, nrand, nreg, model.mreg))

ifp = 0

for isub in range(nsubs):
    print(f"Subject {isub}: ", end='')
    for irand in range(nrand):
        print(f"{irand}", end=' ', flush=True)
        
        for us in [-1, 0, 1]:
            treg = np.random.normal(params.thetareg[isub,:,:,0], params.thetareg[isub,:,:,1])
            tsub = np.zeros(1)
            thetareg[isub,irand,:,:] = treg

            fps, stability, eigvals, eigvecs = dsa.find_fixed_points_network(model, ds.w[isub], treg, tsub,
                                                                             us=us, n=100, init_range=(-2,2))

            ts, xs, converging, periodic, period = dsa.find_attractors_network(model, ds.w[isub], treg, tsub,
                                                                               us=us, n=100, init_range=(-2,2), 
                                                                               T1=400, T2=1200)

            for i, fp in enumerate(fps):    
                rows_fp.append(dict(isub=isub, irand=irand, us=us, n=i, ifp=ifp, stable=stability[i]))
                fixed_points.append(fps[i])
                eigenvalues.append(eigvals[i])        
                ifp += 1

            rows_at.append(dict(isub=isub, irand=irand, us=us,
                                has_converging=np.any(converging),
                                has_nonconverging=np.any(~converging),
                                has_periodic=np.any(periodic[~converging]),
                                has_aperiodic=np.any(~periodic[~converging]),
                               ))
    print("")

df_fp = pd.DataFrame(rows_fp)
df_at = pd.DataFrame(rows_at)
fixed_points = np.array(fixed_points)
eigenvalues = np.array(eigenvalues)    

df_fp.to_csv("./res/subj_fp2.csv")
df_at.to_csv("./res/subj_at2.csv")
np.savez("./res/subj_fp2.npz", fixed_points=fixed_points, eigenvalues=eigenvalues, thetareg=thetareg)

Subject 0: 0 
Subject 1: 0 
Subject 2: 0 
Subject 3: 0 
Subject 4: 0 
Subject 5: 0 
Subject 6: 0 
Subject 7: 0 
Subject 8: 0 
Subject 9: 0 
Subject 10: 0 
Subject 11: 0 
Subject 12: 0 
Subject 13: 0 
Subject 14: 0 
Subject 15: 0 
Subject 16: 0 
Subject 17: 0 
Subject 18: 0 
Subject 19: 0 
Subject 20: 0 
Subject 21: 0 
Subject 22: 0 
Subject 23: 0 
Subject 24: 0 
Subject 25: 0 
Subject 26: 0 
Subject 27: 0 
Subject 28: 0 
Subject 29: 0 
Subject 30: 0 
Subject 31: 0 
Subject 32: 0 
Subject 33: 0 
Subject 34: 0 
Subject 35: 0 
Subject 36: 0 
Subject 37: 0 
Subject 38: 0 
Subject 39: 0 
Subject 40: 0 
Subject 41: 0 
Subject 42: 0 
Subject 43: 0 
Subject 44: 0 
Subject 45: 0 
Subject 46: 0 
Subject 47: 0 
Subject 48: 0 
Subject 49: 0 
Subject 50: 0 
Subject 51: 0 
Subject 52: 0 
Subject 53: 0 
Subject 54: 0 
Subject 55: 0 
Subject 56: 0 
Subject 57: 0 
Subject 58: 0 
Subject 59: 0 
Subject 60: 0 
Subject 61: 0 
Subject 62: 0 
Subject 63: 0 
Subject 64: 0 
Subject 65: 0 
Subject 66: 0 
Subje

In [52]:
## Add u

import itertools

df_fp['us'] = 0.

uss = itertools.cycle([-1, 0, 1])
us = None

lastn = 0
for i in range(len(df_fp)):
    if df_fp.loc[i, 'n'] <= lastn: 
        us = next(uss)
    df_fp.loc[i, 'us'] = us    
    lastn = df_fp.loc[i, 'n']

In [59]:
df_fp = pd.read_csv("./res/subj_fp.csv")
df_at = pd.read_csv("./res/subj_at.csv")
data = np.load("./res/subj_fp.npz")
fixed_points = data['fixed_points']
eigenvalues = data['eigenvalues']
thetareg = data['thetareg']

df_fp['us'] = 0.

## Multistable configurations ?

In [60]:
for isub in range(nsub):
    for irand in range(4):
        for us in [-1, 0, 1]:
            dff = df_fp[(df_fp.isub == isub) & (df_fp.irand == irand) & (df_fp.us == us)]
            if np.any(dff.n >= 1) or np.any(~dff.stable):            
                print(f"Subject {isub}, init {irand}, us = {us}")

                for i, (_, row) in enumerate(dff.iterrows()):
                    print(f"    FP{i}: {'Stable' if row.stable else 'Unstable'}")

                ifps = dff.ifp.to_list()
                for i, ifp1 in enumerate(ifps):
                    for j, ifp2 in enumerate(ifps[:i]):
                        x1 = fixed_points[ifp1]
                        x2 = fixed_points[ifp2]
                        l2 = np.linalg.norm(x1-x2)
                        linf = np.linalg.norm(x1-x2, ord=np.inf)
                        print(f"    Distance FP{j}-FP{i}: L2 {l2:8.5f}   Linf {linf:8.5f}")

                print("")

Subject 3, init 0, us = 0
    FP0: Stable
    FP1: Unstable
    FP2: Stable
    Distance FP0-FP1: L2  0.05016   Linf  0.05369
    Distance FP0-FP2: L2  0.05056   Linf  0.05427
    Distance FP1-FP2: L2  0.00065   Linf  0.00058

Subject 11, init 3, us = 0
    FP0: Stable
    FP1: Stable
    FP2: Unstable
    Distance FP0-FP1: L2  0.13174   Linf  0.10531
    Distance FP0-FP2: L2  0.07230   Linf  0.05757
    Distance FP1-FP2: L2  0.05984   Linf  0.04774

Subject 45, init 1, us = 0
    FP0: Stable
    FP1: Stable
    Distance FP0-FP1: L2  0.17926   Linf  0.14978

Subject 45, init 2, us = 0
    FP0: Stable
    FP1: Stable
    FP2: Unstable
    Distance FP0-FP1: L2  0.17318   Linf  0.15170
    Distance FP0-FP2: L2  0.08815   Linf  0.07504
    Distance FP1-FP2: L2  0.08545   Linf  0.07855

Subject 45, init 3, us = 0
    FP0: Stable
    FP1: Stable
    FP2: Unstable
    Distance FP0-FP1: L2  0.65496   Linf  0.57131
    Distance FP0-FP2: L2  0.08155   Linf  0.08007
    Distance FP1-FP2: L2  0.57

In [56]:
df_fp[df_fp.isub == 91]

,isub,irand,n,ifp,stable,us
276,91,0,0,276,True,-1.0
277,91,0,0,277,False,0.0
278,91,0,1,278,True,0.0
279,91,0,2,279,True,0.0
280,91,0,0,280,True,1.0


## Attractors?

In [9]:
df_at

,isub,irand,us,has_converging,has_nonconverging,has_periodic,has_aperiodic
0,0,0,-1,True,False,False,False
1,0,0,0,True,False,False,False
2,0,0,1,True,False,False,False
3,1,0,-1,True,False,False,False
4,1,0,0,True,False,False,False
...,...,...,...,...,...,...,...
295,98,0,0,True,False,False,False
296,98,0,1,True,False,False,False
297,99,0,-1,True,False,False,False
298,99,0,0,True,False,False,False


In [10]:
print(f"Converging:    {sum(df_at.has_converging)} / {len(df_at)}")
print(f"Nonconverging: {sum(df_at.has_nonconverging)} / {len(df_at)}")
print(f"Periodic:      {sum(df_at.has_periodic)} / {len(df_at)}")
print(f"Aperiodic:     {sum(df_at.has_aperiodic)} / {len(df_at)}")

Converging:    300 / 300
Nonconverging: 0 / 300
Periodic:      0 / 300
Aperiodic:     0 / 300
